## nbjob.structs - Structs for Jupyter notebooks

Lets you define methods in a different cell from the main class.
Also lets you add new methods to existing instances of a class.

Scroll down to "Examples" for documentation

In [2]:
from types import MethodType

In [3]:
class Struct(type):    
    def __new__(cls, name, bases, dct):
        dct['__getattr__'] = cls.getattr
        dct['__setattr__'] = cls.setattr
        
        dct['_methods'] = {}
        dct['_initializers'] = {}
        
        return super(Struct, cls).__new__(cls, name, bases, dct)
    
    def getattr(self, name):
        cls = type(self)
        
        # Methods
        if name in cls._methods:
            return MethodType(cls._methods[name], self)
        
        # Variables (if not already initialized)
        if name in self._initializers:
            initializer, result_set = self._initializers[name]
            initializer(self)
            for other_name in result_set:
                if other_name == name:
                    continue
                try:
                    object.__getattribute__(self, other_name)
                except AttributeError:
                    raise AttributeError("Initializer for {} failed to initialize {}".format(name, other_name))
                    
        
        return object.__getattribute__(self, name)
    
    def setattr(self, name, value):
        cls = type(self)
        
        return object.__setattr__(self, name, value)
    
    # Maybe TODO: method to generate a class that does away with the indirection

In [4]:
def impl(cls):
    def deco(fn):
        assert isinstance(cls, Struct)
        cls._methods[fn.__name__] = fn        
        return fn
    return deco

In [5]:
class VarCollector(object):
    def __init__(self):
        self._variables = set()
        
    def __setattr__(self, name, value):
        if name != '_variables':
            self._variables.add(name)
        
        return object.__setattr__(self, name, value)

def init(cls, vars=None):
    def deco(fn):
        nonlocal vars
        assert isinstance(cls, Struct)
        if vars is None:
            # Get variables set from a dummy
            # (assumes initializer takes no arguments other than self)
            dummy = VarCollector()
            fn(dummy)
            vars = dummy._variables
        
        if not vars:
            raise Exception("Initializer does not set any member variables")
        
        vars = set(vars)
        for var in vars:
            cls._initializers[var] = fn, vars
        
        return fn
    return deco

In [6]:
# The magic string below signals to the notebook importing machinery that
# the examples section below should not be imported into any other modules

#NBIMPORT_STOP

# Examples

In [7]:
class Test(metaclass=Struct):
    pass

In [8]:
t = Test()

In [9]:
t.foo()

AttributeError: 'Test' object has no attribute 'foo'

In [13]:
t.counter

AttributeError: 'Test' object has no attribute 'counter'

In [11]:
@impl(Test)
def foo(self):
    print(self)

In [12]:
t.foo()

In [14]:
@init(Test)
def _(self):
    self.counter = 0

@impl(Test)
def bar(self, inc):
    self.counter += inc
    return self.counter

In [15]:
t.bar(2)

2

In [16]:
t.bar(5)

7

In [17]:
t.counter

7